In [1]:
# get some hybsets 
from os import listdir
from os.path import isfile, join
import yaml
from tqdm import tqdm

# load data (examples of chrony)
hyb_dir = "/home/ubuntu/praxi/fall2019/data/version_hybsets"
hybset_fnames = [f for f in listdir(hyb_dir) if isfile(join(hyb_dir, f))]

hyb_dics = [] 
for hyb_name in tqdm(hybset_fnames):
        full_hyb_name = hyb_dir + '/' + hyb_name
        with open(full_hyb_name, 'r') as stream:
            hyb_dic = yaml.safe_load(stream)
        hyb_dics.append(hyb_dic)
        
print(hyb_dics[0])

100%|██████████| 1300/1300 [01:28<00:00, 14.62it/s]

{'changes': ['/usr/share/man/man1/rrdtool.1.gz', '/usr/share/man/man3/librrd.3.gz', '/usr/share/doc/rrdtool/html/rrdgraph_libdbi.html', '/usr/share/doc/rrdtool/examples/perftest.pl.gz', '/usr/share/man/man1/cdeftutorial.1.gz', '/usr/share/doc/rrdtool/html/rrdthreads.html', '/var/cache/man/de/31234', '/usr/share/doc/rrdtool/html/rrdlastupdate.html', '/usr/share/doc/rrdtool/html/rpntutorial.html', '/usr/share/man/man1/rrdcreate.1.gz', '/usr/share/man/man1/rrdgraph_examples.1.gz', '/usr/share/man/man1/rrdcgi.1.gz', '/var/cache/man/sv/31234', '/usr/bin/rrdtool', '/usr/share/doc/rrdtool.dpkg-new/html/rrdfetch.html', '/usr/share/man/man1/rrdflushcached.1.gz', '/var/cache/man/pl/31234', '/usr/share/doc/rrdtool/copyright', '/usr/bin/rrdinfo.dpkg-new', '/usr/share/doc/rrdtool.dpkg-new/html/rrdgraph_examples.html', '/usr/share/doc/rrdtool.dpkg-new/copyright', '/usr/share/doc/rrdtool.dpkg-new/html/rrdinfo.html', '/usr/share/man/man1/rrdfetch.1.gz', '/var/lib/dpkg/tmp.ci/control', '/usr/share/man/

In [5]:
# for one tagset, get most frequent tag
dic_tags = hyb_dics[0]['tags']
tag_freq_list = []
for tag in dic_tags:
    tag_split = tag.split(':')
    tag_freq_list.append((tag_split[0], int(tag_split[1])))
print(tag_freq_list[0])

140


In [7]:
from operator import itemgetter
#data = [('abc', 121),('abc', 231),('abc', 148), ('abc',221)]
sorted_list = sorted(tag_freq_list,key=itemgetter(1), reverse=True)
print(sorted_list[0::3])

('rrd', 215)


In [10]:
print(sorted_list[0:5])

[('rrd', 215), ('man', 140), ('rrdt', 121), ('rrdtool.', 58), ('rrdgraph', 24)]


In [12]:
chrony_dir = "/home/ubuntu/praxi/fall2019/data/prelim_data"
chrony_fnames = [f for f in listdir(chrony_dir) if isfile(join(chrony_dir, f))]

chrony_dics = [] 
for chrony_name in tqdm(chrony_fnames):
        full_chrony_name = chrony_dir + '/' + chrony_name
        with open(full_chrony_name, 'r') as stream:
            chrony_dic = yaml.safe_load(stream)
        chrony_dics.append(chrony_dic)
        
changes = []
labels = []
for dic in chrony_dics:
    changes.append(dic["changes"])
    labels.append(dic["version"])
    
print(changes[0])

100%|██████████| 30/30 [00:01<00:00, 21.65it/s]

['/var/cache/debconf/templates.dat-new', '/var/cache/man/hu/21009', '/var/lib/dpkg/updates/0007', '/usr/share/man/man1/chrony.1.gz.dpkg-new', '/etc/ppp/ip-down.d/chrony.dpkg-new', '/var/cache/man/pt_BR/21009', '/var/cache/man/fi/21009', '/etc/cron.weekly/chrony.dpkg-new', '/usr/share/doc/chrony/faq.txt.gz.dpkg-new', '/usr/sbin/chronyd', '/var/cache/debconf/templates.dat-old', '/var/lib/dpkg/info/chrony.list-new', '/var/lib/dpkg/updates/0005', '/usr/share/doc/chrony/chrony.txt.gz.dpkg-new', '/usr/share/man/man8/chronyd.8.gz', '/usr/share/doc/chrony.dpkg-new/README.Debian', '/usr/share/chrony.dpkg-new/chrony.conf', '/var/lib/dpkg/tmp.ci/prerm', '/var/lib/dpkg/updates/0001', '/var/lib/dpkg/info/chrony.postinst', '/usr/share/doc/chrony/chrony.html', '/usr/share/doc/chrony/copyright', '/var/cache/man/gl/21009', '/var/lib/dpkg/lock', '/var/log/dpkg.log', '/usr/share/man/man1/chrony.1.gz', '/var/lib/dpkg/tmp.ci/md5sums', '/var/log/apt/history.log', '/var/lib/dpkg/available', '/var/cache/man/i

In [17]:
# trying a different rule-based method 
from collections import OrderedDict
from orderedset import OrderedSet
import yaml

def fit(X, y): # X is list of lists, little lists have tags, y are VERSIONS
    label_to_tokens = transform_tagsets(X, y)
    labels = label_to_tokens.keys()
    #print(label_to_tokens)
    #input("Enter to continue...")
    # get inverse map
    rules = {}
    #for l in labels: 
    #    rules[l]=[]
    token_to_labels = get_token_to_labels(label_to_tokens)
    for token in token_to_labels.keys():
        if len(token_to_labels[token]) == 1:
            if token_to_labels[token][0] not in rules: # only take one rule 
                rules[token_to_labels[token][0]] = token  
    return rules

def transform_tagsets(tagsets, labels, take_max=False):  # Changesets as dictionaries
    res = OrderedDict()
    for data, label in zip(tagsets, labels):
        for token in data:
            if label not in res:
                res[label] = OrderedDict()
            if token not in res[label]:
                res[label][token] = 1
            else:
                res[label][token] += 1
    newres = dict()
    for label in res:
        newres[label] = set()
        maxval = max(res[label].values())
        for token in sorted(res[label], key=res[label].get, reverse=True):
            if res[label][token] < (maxval - 6):
                break
            newres[label].add(token)
    return newres

def get_token_to_labels(label_to_tokens):
    """ Returns inverse map: from tokens to sets of labels """
    token_to_labels = OrderedDict()
    for label in label_to_tokens:
        for token in label_to_tokens[label]:
            if token not in token_to_labels:
                token_to_labels[token] = OrderedSet()
            token_to_labels[token].add(label)
    return token_to_labels

In [18]:
rules = fit(changes, labels)

In [19]:
print(rules)

{'1.29-1ubuntu0.1': '/usr/share/doc/chrony/chrony.txt.gz.dpkg-new', '2.1.1-1ubuntu0.1': '/var/log/chrony/statistics.log', '3.2-4ubuntu4.2': '/var/cache/man/da/CACHEDIR.TAG'}
